# MIE1621 Fall 2017 Project
## Jonathan Smith 999694521

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy as sci
from numpy.linalg import inv,norm
from scipy.optimize import minimize

### Data Initialization & Pre-Processing
Create functions that will convert a csv file into covariance and returns

In [2]:
### Manually input stock data
# stocks = pd.DataFrame(columns=['Return', 'var_1', 'var_2', 'var_3'],
#                    data=[[0.1073,0.02778,0.00387,0.00021],
#                         [0.0731,0.00387,0.01112,-0.0002],
#                         [0.0621,0.00021,-0.0002,0.00115]]
#                   )
# stocks.to_csv('stock_data.csv')

In [3]:
stocks = pd.read_csv('stock_data.csv')

In [4]:
def split (df):
    '''Function to split input dataframe into variance and return'''
    ret=df.loc[:,'Return'].as_matrix()
    var=df.iloc[:,2:].as_matrix()
    return ret, var

In [5]:
def kkt_convert (df,d=3.5):
    '''Function to split input dataframe into variance and return
    and convert them into kkt matrices'''
    ret=df.loc[:,'Return'].as_matrix()
    ret = np.append(ret,1)
    
    var=df.iloc[:,2:].as_matrix()
    var=d*var
    size=len(var)
    var = np.insert(var,size,1,axis=1)
    var = np.insert(var,size,1,axis=0)
    var[size][size]=0
    return ret, var 

In [6]:
ret_kkt, var_kkt = kkt_convert(stocks)

In [7]:
ret_kkt

array([ 0.1073,  0.0731,  0.0621,  1.    ])

In [8]:
var_kkt

array([[  9.72300000e-02,   1.35450000e-02,   7.35000000e-04,
          1.00000000e+00],
       [  1.35450000e-02,   3.89200000e-02,  -7.00000000e-04,
          1.00000000e+00],
       [  7.35000000e-04,  -7.00000000e-04,   4.02500000e-03,
          1.00000000e+00],
       [  1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
          0.00000000e+00]])

In [9]:
ret, var = split(stocks)

In [10]:
ret

array([ 0.1073,  0.0731,  0.0621])

In [11]:
var

array([[ 0.02778,  0.00387,  0.00021],
       [ 0.00387,  0.01112, -0.0002 ],
       [ 0.00021, -0.0002 ,  0.00115]])

### Define Functions to Evaluate Objective Function & Derivatives

In [12]:
def f(x, d=3.5):
    '''Return value of function'''
#     return np.dot(x,ret)-np.dot(0.5*d*np.matmul(np.transpose(x),var),x)
    return np.dot(x,ret)-np.dot(0.5*d*np.matmul(x,var),x)

In [13]:
def df_kkt(x): 
    '''return value of kkt gradient'''
    return np.matmul(var_kkt,x)-ret_kkt

In [14]:
df_kkt([1,1,1,1])

array([ 1.00421 ,  0.978665,  0.94196 ,  2.      ])

In [15]:
# minimize(f,[1,1,1],method='BFGS')

In [16]:
# def d_f(x,d=4):
#     return ret-d*np.matmul(var,x)

In [17]:
# def d2_f(x,d=4):
#     return -d*var

In [18]:
f([1,1,1])

0.15883249999999999

### Define Functions for Newton, SDM, BFGS Methods

In [19]:
x=[1,1,1,1]
alpha=1
d=np.matmul(inv(var_kkt),df_kkt(x)) # From definition of newton
x_k = x - alpha*d
Rho=0.5
gamma=0.0001

print(f(x[:-1]))
print(f(x[:-1])+alpha*gamma*np.matmul(d,df_kkt(x)))

0.1588325
0.159215233116


In [20]:
def newton (df,d,init,tol=0.001,limit=100, backtrack=False, Rho=0.5, gamma=0.01):
    '''
    Perform Newton Method to find optimal solution for some function f
    df-dataframe containing return and covariance
    d-risk tolerance factor
    init-initial guess
    tol-stopping tolerance between interations
    limit-maximum number of iterations
    backtrack-perform backtrack line search if true, use step length 1 otherwise
    Rho,gamma-backtrack parameters
    '''
    ret,var=split(df)
    ret_kkt, var_kkt = kkt_convert(df)
    x=init
    rho=Rho
    delta=[tol]
    i=0
    print('Initial guess: ',x)
    while (np.any(np.absolute(delta)>=tol))&(i<limit):
        alpha=1
        d=np.matmul(inv(var_kkt),df_kkt(x)) # From definition of newton
        x_k = x - alpha*d
        while (alpha>0.01)&(backtrack==True)&(f(x_k[:-1])>(f(x[:-1])+alpha*gamma*np.matmul(d,df_kkt(x)))):
            alpha=alpha*rho
            x_k = x - alpha*d
        if alpha !=1:
            print('alpha: ',alpha)
        x_k = x - alpha*d
        delta = x_k-x
        x=x_k
        i+=1
        print('Iteration',i,':',x)
    print('slope:',df_kkt(x))
    x=x[0:-1]
    print('Solution: x =',x,', Return:',f(x))

In [21]:
def steepest_descent (df,d,init,tol=0.001,limit=100, backtrack=False, rho=0.5, gamma=0.01):
    '''
    Perform Steepest Descent Method to find optimal solution for some function f
    df-dataframe containing return and covariance
    d-risk tolerance factor
    init-initial guess
    tol-stopping tolerance between interations
    limit-maximum number of iterations
    backtrack-perform backtrack line search if true, use step length 1 otherwise
    Rho,gamma-backtrack parameters
    '''
    ret,var=split(df)
    ret_kkt, var_kkt = kkt_convert(df)
    x=init
    x_k=x
    delta=[tol]
    i=0
    print('Initial guess: ',x)
#     while (np.any(np.absolute(delta)>=tol))&(np.all(df_kkt(x)<0))&(i<limit):
    while (np.any(np.absolute(delta)>=tol))&(f(x_k[:-1])>=f(x[:-1]))&(i<limit):
        alpha=1
        d=df_kkt(x) # From definition of steepest descent
        x_k = x - alpha*d
        while (alpha>0.05)&(backtrack==True)&((f(x_k[:-1]))>(f(x[:-1])+alpha*gamma*np.matmul(d,(df_kkt(x))))):
            alpha=alpha*rho
            x_k = x - alpha*d
        if alpha !=1:
            print('alpha: ',alpha)
        x_k = x-alpha*d
        delta = x_k-x
        x=x_k
        i+=1
        print('Iteration',i,':',x)
        print('slope:',df_kkt(x))
    print('slope:',df_kkt(x))
    x=x[0:-1]
    print('Solution: x =',x,', Return:',f(x))

In [22]:
def BFGS (df,d,init,tol=0.001,limit=100, backtrack=False, rho=0.5, gamma=0.0001):
    '''
    Perform BFGS Quasi-Newton Method to find optimal solution for some function f
    '''
    ret,var=split(df)
    ret_kkt, var_kkt = kkt_convert(df)
    x=init
    H=inv(np.eye(1+len(var)))
    I=np.eye(1+len(var))
    delta=[tol]
    i=0
    print('Initial guess: ',x)
    while (np.any(np.absolute(delta)>=tol))&(i<limit):
        alpha=1
        d=np.matmul(np.asarray(H),df_kkt(x))
        x_k = x - alpha*d
        while (alpha>0.1)&(backtrack==True)&(f(x_k[:-1])>(f(x[:-1])+alpha*gamma*np.matmul(d,df_kkt(x)))):
            alpha=alpha*rho
            x_k = x - alpha*d
        if alpha !=1:
            print('alpha: ',alpha)
        x_k = x - alpha*d
        delta = x_k-x
#         find udpated H inverse
        s=np.asmatrix(x_k-x)
        y=np.asmatrix(df_kkt(x_k)-df_kkt(x))
        a=np.asscalar(1/(y*np.transpose(s)))
        A=a*(np.transpose(s)*y)
        B=a*(np.transpose(y)*s)
        C=a*(np.transpose(s)*(s))
        H=np.matmul(np.matmul((I-A),H),(I-B))+C 
        x=x_k
        i+=1
        print('Iteration',i,':',x)
    print('slope:',df_kkt(x))
    x=x[0:-1]
    print('Solution: x =',x,', Return:',f(x))

### Perform 3 Methods with Step Length = 1

In [23]:
%time
ret, var = split(stocks)
ret_kkt, var_kkt = kkt_convert(stocks)
newton(stocks,4,[1,1,1,1])

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.77 µs
Initial guess:  [1, 1, 1, 1]
Iteration 1 : [ 0.45526555  0.1745838   0.37015065  0.06039773]
Iteration 2 : [ 0.45526555  0.1745838   0.37015065  0.06039773]
slope: [ 0.  0.  0.  0.]
Solution: x = [ 0.45526555  0.1745838   0.37015065] , Return: 0.0724980784211


In [24]:
%time
ret, var = split(stocks)
ret_kkt, var_kkt = kkt_convert(stocks)
# steepest_descent(stocks,4,[0.4,0.1,0.4,0.01])
steepest_descent(stocks,4,[1,1,1,1])

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs
Initial guess:  [1, 1, 1, 1]
Iteration 1 : [-0.00421   0.021335  0.05804  -1.      ]
slope: [-1.1073777  -1.07236729 -1.06188442 -0.924835  ]
Iteration 2 : [ 1.1031677   1.09370229  1.11992442 -0.075165  ]
slope: [-0.05956666 -0.09153965 -0.13271207  2.31679441]
Iteration 3 : [ 1.16273436  1.18524194  1.25263649 -2.39195941]
slope: [-2.36923196 -2.4040574  -2.44899261  2.60061279]
Iteration 4 : [ 3.53196632  3.58929934  3.70162909 -4.99257219]
slope: [-4.70512135 -4.88072732 -5.03968965  9.82289475]
Iteration 5 : [  8.23708767   8.47002666   8.74131874 -14.81546694]
slope: [-14.00072353 -14.45346108 -14.8422579   24.44843308]
Iteration 6 : [ 22.2378112   22.92348774  23.58357664 -39.26390002]
slope: [-36.88118507 -38.16011523 -39.23077777  67.74487558]
Iteration 7 : [  59.11899627   61.08360297   62.81435441 -107.0087756 ]
slope: [-100.49438964 -103.94770501 -106.81735388  182.01695365]
Iteration 8 : [ 159.6133859   165.03

In [25]:
%time
ret, var = split(stocks)
ret_kkt, var_kkt = kkt_convert(stocks)
BFGS(stocks,4,[1,1,1,1])

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.11 µs
Initial guess:  [1, 1, 1, 1]
Iteration 1 : [-0.00421   0.021335  0.05804  -1.      ]
Iteration 2 : [ 0.60147834  0.60766434  0.6395555  -0.5334302 ]
Iteration 3 : [ 2.01458584  2.00211579  2.05697165 -1.36729661]
Iteration 4 : [ 0.37926823  0.36520077  0.38977057  0.08642783]
Iteration 5 : [ 0.33674282  0.31035651  0.33944003  0.05852694]
Iteration 6 : [ 0.34756647  0.30907218  0.34196769  0.06184588]
Iteration 7 : [ 0.37033821  0.29716183  0.34134922  0.06434301]
Iteration 8 : [ 0.41764051  0.26367232  0.33493663  0.06539792]
Iteration 9 : [ 0.45653346  0.22881726  0.32619087  0.06305928]
Iteration 10 : [ 0.46733115  0.21358531  0.32220453  0.06030821]
Iteration 11 : [ 0.46509935  0.21197629  0.32282827  0.0594258 ]
Iteration 12 : [ 0.46282827  0.21104822  0.3247461   0.05912969]
Iteration 13 : [ 0.45806373  0.2065993   0.33197519  0.05876375]
Iteration 14 : [ 0.45312737  0.19669375  0.34563295  0.05873498]
Iteration 15 :

### Perform 3 Methods with Backtracking

In [26]:
%time
ret, var = split(stocks)
ret_kkt, var_kkt = kkt_convert(stocks)
newton(stocks,4,[1,1,1,1], backtrack=True)

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 3.81 µs
Initial guess:  [1, 1, 1, 1]
Iteration 1 : [ 0.45526555  0.1745838   0.37015065  0.06039773]
alpha:  0.03125
Iteration 2 : [ 0.45526555  0.1745838   0.37015065  0.06039773]
slope: [  8.32667268e-17  -2.22044605e-16  -1.17961196e-16  -2.66453526e-15]
Solution: x = [ 0.45526555  0.1745838   0.37015065] , Return: 0.0724980784211


In [27]:
%time
ret, var = split(stocks)
ret_kkt, var_kkt = kkt_convert(stocks)
steepest_descent(stocks,4,[1,1,1,1], backtrack=True, tol=0.0001)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.15 µs
Initial guess:  [1, 1, 1, 1]
Iteration 1 : [-0.00421   0.021335  0.05804  -1.      ]
slope: [-1.1073777  -1.07236729 -1.06188442 -0.924835  ]
alpha:  0.03125
Iteration 2 : [ 0.03039555  0.05484648  0.09122389 -0.97109891]
slope: [-1.0746336  -1.04171643 -1.03284778 -0.82353408]
alpha:  0.03125
Iteration 3 : [ 0.06397785  0.08740012  0.12350038 -0.94536347]
slope: [-1.04516829 -1.01428172 -1.00698053 -0.72512165]
alpha:  0.03125
Iteration 4 : [ 0.09663936  0.11909642  0.15496852 -0.92270341]
slope: [-1.01888011 -0.98996768 -0.984192   -0.62929569]
alpha:  0.03125
Iteration 5 : [ 0.12847937  0.15003291  0.18572452 -0.90303792]
slope: [-0.99567717 -0.9686884  -0.96440097 -0.5357632 ]
alpha:  0.03125
Iteration 6 : [ 0.15959428  0.18030442  0.21586205 -0.88629532]
slope: [-0.97547709 -0.95036728 -0.94753539 -0.44423925]
alpha:  0.03125
Iteration 7 : [ 0.19007794  0.2100034   0.24547253 -0.87241285]
slope: [-0.95820665 -0.934936

In [28]:
%time
ret, var = split(stocks)
ret_kkt, var_kkt = kkt_convert(stocks)
BFGS(stocks,4,[1,1,1,1], backtrack=True, tol=0.001)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs
Initial guess:  [1, 1, 1, 1]
Iteration 1 : [-0.00421   0.021335  0.05804  -1.      ]
alpha:  0.0625
Iteration 2 : [ 0.03364552  0.05798058  0.09438472 -0.97083939]
alpha:  0.0625
Iteration 3 : [ 0.15745429  0.17948903  0.2170464  -0.99561796]
Iteration 4 : [ 8.16394517  7.93417629  7.99447414  1.18320926]
alpha:  0.0625
Iteration 5 : [ 7.69597509  7.47886257  7.53735717  1.10615406]
alpha:  0.0625
Iteration 6 : [ 7.24234883  7.0365261   7.09340237  1.03963153]
alpha:  0.0625
Iteration 7 : [ 6.81304898  6.61710858  6.67268054  0.97826963]
alpha:  0.0625
Iteration 8 : [ 6.4099826   6.22258186  6.27717223  0.92093946]
alpha:  0.0625
Iteration 9 : [ 6.03227662  5.85218358  5.90609529  0.86722678]
alpha:  0.0625
Iteration 10 : [ 5.67849293  5.50458941  5.55810096  0.81687189]
alpha:  0.0625
Iteration 11 : [ 5.34715007  5.17842492  5.23179081  0.76965829]
alpha:  0.0625
Iteration 12 : [ 5.03683185  4.8723725   4.92582472  0.7253

### Scale up problem

In [29]:
# Read larger dataset into notebook
stocks_2 = pd.read_csv('stock_data_2.csv')
stocks_2

,Unnamed: 0,Return,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10
0,0,0.1465,0.011839,0.008182,0.009835,0.003280,0.007280,0.017415,-0.001371,0.007059,-0.013184,0.010996
1,1,0.9031,0.008182,0.007063,0.008208,0.003968,0.005458,0.013015,-0.000601,0.005886,-0.010116,0.008682
2,2,0.0769,0.009835,0.008208,0.011039,0.003828,0.007353,0.017705,-0.001448,0.006399,-0.012805,0.011075
3,3,0.0940,0.003280,0.003968,0.003828,0.034111,0.001837,0.004495,0.000287,0.004281,-0.005437,0.003056
4,4,0.2831,0.007280,0.005458,0.007353,0.001837,0.005797,0.013893,-0.001743,0.005104,-0.009706,0.007397
5,5,0.2056,0.017415,0.013015,0.017705,0.004495,0.013893,0.034623,-0.004600,0.013692,-0.024277,0.017413
6,6,0.1037,-0.001371,-0.000601,-0.001448,0.000287,-0.001743,-0.004600,0.001642,-0.001503,0.002834,-0.000858
7,7,0.0024,0.007059,0.005886,0.006399,0.004281,0.005104,0.013692,-0.001503,0.015276,-0.012211,0.005652
8,8,0.0270,-0.013184,-0.010116,-0.012805,-0.005437,-0.009706,-0.024277,0.002834,-0.012211,0.019165,-0.012049
9,9,0.8350,0.010996,0.008682,0.011075,0.003056,0.007397,0.017413,-0.000858,0.005652,-0.012049,0.013524


In [30]:
start=[1,1,1,1,1,1,1,1,1,1,1]

In [31]:
%time
ret, var = split(stocks_2)
ret_kkt, var_kkt = kkt_convert(stocks_2)
newton(stocks_2,4,start)

CPU times: user 1 µs, sys: 1 µs, total: 2 µs
Wall time: 4.05 µs
Initial guess:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Iteration 1 : [ -1.85744335e+02   4.94804173e+02  -3.72157226e+02  -6.20465663e+00
   2.00182146e+02  -4.07154733e+01  -1.73055470e+02  -2.93378259e+01
   6.62153162e+00   1.06607136e+02   3.34417130e-02]
Iteration 2 : [ -1.85744335e+02   4.94804173e+02  -3.72157226e+02  -6.20465663e+00
   2.00182146e+02  -4.07154733e+01  -1.73055470e+02  -2.93378259e+01
   6.62153162e+00   1.06607136e+02   3.34417130e-02]
slope: [ -3.33066907e-16   1.11022302e-16  -7.77156117e-16  -3.74700271e-16
  -2.22044605e-16   1.13797860e-15   1.24900090e-16  -4.23706209e-16
  -1.08940634e-15  -8.88178420e-16   0.00000000e+00]
Solution: x = [-185.74433485  494.80417341 -372.1572263    -6.20465663  200.18214607
  -40.71547327 -173.05546963  -29.33782593    6.62153162  106.60713551] , Return: 254.978679003


In [32]:
%time
ret, var = split(stocks_2)
ret_kkt, var_kkt = kkt_convert(stocks_2)
steepest_descent(stocks_2,4,start)

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 11 µs
Initial guess:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Iteration 1 : [-0.06815599  0.72899516 -0.13726138 -0.09396782  0.13375347 -0.15621545
  0.12945689 -0.17132247  0.29925416  0.60789337 -8.        ]
slope: [-8.13563296 -8.8927644  -8.06544481 -8.10094282 -8.27820818 -8.19763124
 -8.09981748 -8.00925046 -8.02995438 -8.81547989  0.27242994]
Iteration 2 : [ 8.06747697  9.62175956  7.92818343  8.006975    8.41196166  8.04141579
  8.22927437  7.83792799  8.32920854  9.42337326 -8.27242994]
slope: [ -6.60733035  -7.7051028   -6.54101057  -6.82945753  -7.29811642
  -5.43749051  -8.58756543  -6.83791889 -10.58117177  -7.17862531
  82.89755656]
Iteration 3 : [ 14.67480733  17.32686237  14.469194    14.83643253  15.71007807
  13.47890629  16.81683979  14.67584688  18.91038031  16.60199857
 -91.1699865 ]
slope: [ -88.27377811  -89.58681961  -88.20712042  -88.5221986   -89.35923556
  -86.32686214  -91.594567    -88.76351343  -94.9895403

In [33]:
%time
ret, var = split(stocks_2)
ret_kkt, var_kkt = kkt_convert(stocks_2)
BFGS(stocks_2,4,start)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs
Initial guess:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Iteration 1 : [-0.06815599  0.72899516 -0.13726138 -0.09396782  0.13375347 -0.15621545
  0.12945689 -0.17132247  0.29925416  0.60789337 -8.        ]
Iteration 2 : [  2.12694858   5.14678387   1.86258397   2.02341362   2.86402307
   1.87569615   2.74919305   1.7200701    3.26175105   4.70935233
 -28.41552502]
Iteration 3 : [ 0.04789897  0.88646583 -0.02880852 -0.01130164  0.2048785   0.09862718
  0.02394987 -0.10867616 -0.03638134  0.80673366  0.98339647]
Iteration 4 : [-0.22199289  1.44836653 -0.37421569 -0.29065122  0.10441181 -0.14980964
 -0.21706586 -0.50410542 -0.28735543  1.271999    0.07414039]
Iteration 5 : [-0.52824191  2.54619586 -0.80850402 -0.57575284  0.08461125 -0.41221639
 -0.47976268 -1.00401225 -0.57592844  2.2007497  -0.14630971]
Iteration 6 : [ -4.1394474   16.071529    -5.98456593  -3.57013833   0.03530425
  -3.55937394  -3.36829912  -6.75254401  -3.61408

In [40]:
%time
ret, var = split(stocks_2)
ret_kkt, var_kkt = kkt_convert(stocks_2)
newton(stocks_2,4,start,limit=600, backtrack=True, tol=0.01)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs
Initial guess:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
alpha:  0.0078125
Iteration 1 : [-0.45894012  4.8578451  -1.91529083  0.94371362  2.55611052  0.67409787
 -0.35980836  0.76298573  1.04391822  1.82505575  0.99244876]
alpha:  0.0078125
Iteration 2 : [-1.90648226  8.68555079 -4.80780595  0.88786698  4.10006392  0.35074184
 -1.70899321  0.52782314  1.08749332  2.64366574  0.98495652]
alpha:  0.0078125
Iteration 3 : [ -3.34271549  12.48335253  -7.6777233    0.83245664   5.63195519
   0.02991204  -3.04763756   0.29449776   1.13072799   3.45588035
   0.97752281]
alpha:  0.0078125
Iteration 4 : [ -4.76772814  16.25148395 -10.52521941   0.77747919   7.15187855
  -0.28841129  -4.37582375   0.06299523   1.1736249    4.26174953
   0.97014718]
alpha:  0.0078125
Iteration 5 : [ -6.18160788  19.99017683 -13.35046947   0.72293125   8.65992752
  -0.60424771  -5.69363348  -0.16669868   1.21618667   5.06132286
   0.96282917]
alpha:  0.00781

In [35]:
%time
ret, var = split(stocks_2)
ret_kkt, var_kkt = kkt_convert(stocks_2)
steepest_descent(stocks_2,4,start, backtrack=True, tol=0.01)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.25 µs
Initial guess:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Iteration 1 : [-0.06815599  0.72899516 -0.13726138 -0.09396782  0.13375347 -0.15621545
  0.12945689 -0.17132247  0.29925416  0.60789337 -8.        ]
slope: [-8.13563296 -8.8927644  -8.06544481 -8.10094282 -8.27820818 -8.19763124
 -8.09981748 -8.00925046 -8.02995438 -8.81547989  0.27242994]
Iteration 2 : [ 8.06747697  9.62175956  7.92818343  8.006975    8.41196166  8.04141579
  8.22927437  7.83792799  8.32920854  9.42337326 -8.27242994]
slope: [ -6.60733035  -7.7051028   -6.54101057  -6.82945753  -7.29811642
  -5.43749051  -8.58756543  -6.83791889 -10.58117177  -7.17862531
  82.89755656]
Iteration 3 : [ 14.67480733  17.32686237  14.469194    14.83643253  15.71007807
  13.47890629  16.81683979  14.67584688  18.91038031  16.60199857
 -91.1699865 ]
slope: [ -88.27377811  -89.58681961  -88.20712042  -88.5221986   -89.35923556
  -86.32686214  -91.594567    -88.76351343  -94.98954

In [44]:
%time
ret, var = split(stocks_2)
ret_kkt, var_kkt = kkt_convert(stocks_2)
BFGS(stocks_2,4,start, backtrack=True, limit=600, tol=0.001)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs
Initial guess:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Iteration 1 : [-0.06815599  0.72899516 -0.13726138 -0.09396782  0.13375347 -0.15621545
  0.12945689 -0.17132247  0.29925416  0.60789337 -8.        ]
alpha:  0.0625
Iteration 2 : [ 0.06903805  1.00510696 -0.01227105  0.03836852  0.30439532 -0.02922098
  0.2931904  -0.05311044  0.48441022  0.86423455 -9.27597031]
Iteration 3 : [ 0.04789897  0.88646583 -0.02880852 -0.01130164  0.2048785   0.09862718
  0.02394987 -0.10867616 -0.03638134  0.80673366  0.98339647]
alpha:  0.0625
Iteration 4 : [ 0.02496951  0.93598375 -0.05823781 -0.03450191  0.19777211  0.07480623
  0.00799509 -0.14163897 -0.04859945  0.84692537  0.71966439]
alpha:  0.0625
Iteration 5 : [ 0.00210075  1.00100746 -0.08909713 -0.05689643  0.19296488  0.054125
 -0.01142512 -0.17696467 -0.06880162  0.90139238  0.64864297]
alpha:  0.0625
Iteration 6 : [-0.04578995  1.15252652 -0.15516042 -0.10062792  0.18626952  0.01149